# Few shot

### CodeSearchNet

In [1]:
import warnings

# Suppress all DeprecationWarnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
from datasets import load_dataset
dataset = load_dataset('code_search_net', 'python')

In [12]:
sample_size = 200
val_dataset = dataset['validation'].shuffle(seed=40).select(range(sample_size))

In [15]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm import tqdm
from codebleu import calc_codebleu
import torch

model_name = "Qwen/Qwen2.5-7B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Iterate over the validation dataset and generate responses
references = []
candidates = []

for sample in tqdm(val_dataset, desc="Processing samples", unit="sample"):
    prompt = sample['func_documentation_string']
    content = """Give me python code to accomplish this function, Do not include any explanation. 
                Only provide me the code with the function definition.
                For example, for problem 'Filter binding from ISBN record. In MARC XML / OAI, the binding\n        information is stored in same subrecord as ISBN.\n\n        Example:\n            ``<subfield code="a">80-251-0225-4 (brož.) :</subfield>`` ->\n            ``brož.``.', 
                you are supposed to response 'def _filter_binding(self, binding):\n        ###\n        Filter binding from ISBN record. In MARC XML / OAI, the binding\n        information is stored in same subrecord as ISBN.\n\n        Example:\n            ``<subfield code="a">80-251-0225-4 (brož.) :</subfield>`` ->\n            ``brož.``.\n        ###\n        binding = binding.strip().split(" ", 1)[-1]  # isolate bind. from ISBN\n        binding = remove_hairs_fn(binding)  # remove special chars from binding\n\n        return binding.split(":")[-1].strip()'.
                For problem '!\r\n        @brief Returns clustering ordering information about the input data set.\r\n        @details Clustering ordering of data-set contains the information about the internal clustering structure in line with connectivity radius.\r\n        \r\n        @return (ordering_analyser) Analyser of clustering ordering.\r\n        \r\n        @see process()\r\n        @see get_clusters()\r\n        @see get_noise()\r\n        @see get_radius()\r\n        @see get_optics_objects()', 
                you are supposed to response 'def get_ordering(self):\r\n        ###!\r\n        @brief Returns clustering ordering information about the input data set.\r\n        @details Clustering ordering of data-set contains the information about the internal clustering structure in line with connectivity radius.\r\n        \r\n        @return (ordering_analyser) Analyser of clustering ordering.\r\n        \r\n        @see process()\r\n        @see get_clusters()\r\n        @see get_noise()\r\n        @see get_radius()\r\n        @see get_optics_objects()\r\n        \r\n        ###\r\n        \r\n        if self.__ordering is None:\r\n            self.__ordering = []\r\n        \r\n            for cluster in self.__clusters:\r\n                for index_object in cluster:\r\n                    optics_object = self.__optics_objects[index_object]\r\n                    if optics_object.reachability_distance is not None:\r\n                        self.__ordering.append(optics_object.reachability_distance)\r\n            \r\n        return self.__ordering'. 
                For problem '[virtual method override] Emits a single prediction as input versus\n    predicted.\n\n    modelResult:    An opf_utils.ModelResult object that contains the model input\n                    and output for the current timestep.', 
                you are supposed to response 'def append(self, modelResult):\n    ### [virtual method override] Emits a single prediction as input versus\n    predicted.\n\n    modelResult:    An opf_utils.ModelResult object that contains the model input\n                    and output for the current timestep.\n    ###\n\n    #print "DEBUG: _BasicPredictionWriter: writing modelResult: %r" % (modelResult,)\n\n    # If there are no inferences, don\'t write anything\n    inferences = modelResult.inferences\n    hasInferences = False\n    if inferences is not None:\n      for value in inferences.itervalues():\n        hasInferences = hasInferences or (value is not None)\n\n    if not hasInferences:\n      return\n\n    if self.__dataset is None:\n      self.__openDatafile(modelResult)\n\n    inputData = modelResult.sensorInput\n\n    sequenceReset = int(bool(inputData.sequenceReset))\n    outputRow = [sequenceReset]\n\n\n    # -----------------------------------------------------------------------\n    # Write out the raw inputs\n    rawInput = modelResult.rawInput\n    for field in self._rawInputNames:\n      outputRow.append(str(rawInput[field]))\n\n    # -----------------------------------------------------------------------\n    # Write out the inference element info\n    for inferenceElement, outputVal in inferences.iteritems():\n      inputElement = InferenceElement.getInputElement(inferenceElement)\n      if inputElement:\n        inputVal = getattr(inputData, inputElement)\n      else:\n        inputVal = None\n\n      if type(outputVal) in (list, tuple):\n        assert type(inputVal) in (list, tuple, None)\n\n        for iv, ov in zip(inputVal, outputVal):\n          # Write actual\n          outputRow.append(str(iv))\n\n          # Write inferred\n          outputRow.append(str(ov))\n      elif isinstance(outputVal, dict):\n        if inputVal is not None:\n          # If we have a predicted field, include only that in the actuals\n          if modelResult.predictedFieldName is not None:\n            outputRow.append(str(inputVal[modelResult.predictedFieldName]))\n          else:\n            outputRow.append(str(inputVal))\n        for key in sorted(outputVal.keys()):\n          outputRow.append(str(outputVal[key]))\n      else:\n        if inputVal is not None:\n          outputRow.append(str(inputVal))\n        outputRow.append(str(outputVal))\n\n    metrics = modelResult.metrics\n    for metricName in self.__metricNames:\n      outputRow.append(metrics.get(metricName, 0.0))\n\n    #print "DEBUG: _BasicPredictionWriter: writing outputRow: %r" % (outputRow,)\n\n    self.__dataset.appendRecord(outputRow)\n\n    self.__dataset.flush()\n\n    return'"""

    messages = [
        {"role": "system", "content": content + "Now answer the following coding question. "},
        {"role": "user", "content": prompt}
    ]
    
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    # Store references and candidates
    references.append([sample['func_code_string']])  # Wrap reference in a list
    candidates.append(response)

# Compute BLEU scores
# bleu_scores, sentence_scores = calculate_bleu(references, candidates)
bleu_scores = calc_codebleu(references, candidates, "python")

# Print results
print(bleu_scores)
# print("BLEU Scores:", bleu_scores)
# print("Sentence BLEU Scores (First 10):", sentence_scores[:10])  # Print only first 10 for brevity


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Processing samples: 100%|████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [24:36<00:00,  7.38s/sample]
/opt/conda/lib/python3.12/site-packages/tree_sitter/__init__.py:36: FutureWarning: Language(path, name) is deprecated. Use Language(ptr, name) instead.
  warn("{} is deprecated. Use {} instead.".format(old, new), FutureWarning)


{'codebleu': 0.29763634606681016, 'ngram_match_score': 0.2853592888953933, 'weighted_ngram_match_score': 0.31937717643447117, 'syntax_match_score': 0.2777777777777778, 'dataflow_match_score': 0.30803114115959845}
